## 1. Domain Knowledge

This dataset represents **hourly POS (Point of Sale) checkout counts** for a retail or e-commerce operation.

Each CSV contains:
- **time**: The hour of the day (00h to 23h).
- **today**: Number of completed checkouts during that hour on the current day.
- **yesterday**: Number of completed checkouts during the same hour on the previous day.
- **same_day_last_week**: Checkouts during the same hour, on the same weekday, one week ago.
- **avg_last_week**: Average hourly checkouts over the past 7 days for the same hour.
- **avg_last_month**: Average hourly checkouts over the past 30 days for the same hour.

The purpose of this data is to allow **real-time comparison of today's sales performance against historical baselines**. This is a common pattern in operational monitoring dashboards (e.g., Grafana, CloudWatch) where operators compare live metrics to prior periods to quickly spot deviations.

Typical anomaly patterns to watch for:
- **Sudden drops to zero**: Could indicate a system outage (payment gateway failure, server crash, deployment gone wrong).
- **Unexpected spikes**: Could indicate a flash sale, bot activity, or double-counting bug.
- **Gradual drift from averages**: Could indicate a slow degradation, changing user behavior, or data pipeline lag.

We have two snapshots:
- `checkout_1.csv`: A snapshot taken at the end of a presumably normal day.
- `checkout_2.csv`: A snapshot taken at the end of a day that may contain an anomalous event.

## 2. Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import sqlite3

plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.size'] = 12

## 3. Data Loading

Loading both checkout snapshots.

In [2]:
df_c1 = pd.read_csv('sample_data/checkout/checkout_1.csv')
df_c2 = pd.read_csv('sample_data/checkout/checkout_2.csv')

print(f"Checkout 1: {df_c1.shape[0]} rows, {df_c1.shape[1]} columns")
print(f"Checkout 2: {df_c2.shape[0]} rows, {df_c2.shape[1]} columns")
print(f"\nColumns: {list(df_c1.columns)}")

Checkout 1: 24 rows, 6 columns
Checkout 2: 24 rows, 6 columns

Columns: ['time', 'today', 'yesterday', 'same_day_last_week', 'avg_last_week', 'avg_last_month']


### 3.1 Sanity Checks - Checkout 1

In [3]:
print("CHECKOUT 1 SANITY CHECKS")
print("=" * 40)
print(f"Shape: {df_c1.shape}")
print(f"\nNull Values:\n{df_c1.isnull().sum()}")
print(f"\nData Types:\n{df_c1.dtypes}")
print(f"\nExpected 24 hourly rows: {'PASS' if df_c1.shape[0] == 24 else 'FAIL'}")
print(f"Duplicate time entries: {df_c1['time'].duplicated().sum()}")
print(f"Negative values in 'today': {(df_c1['today'] < 0).sum()}")
print(f"Negative values in 'yesterday': {(df_c1['yesterday'] < 0).sum()}")
print(f"Negative values in 'same_day_last_week': {(df_c1['same_day_last_week'] < 0).sum()}")
print(f"Negative values in 'avg_last_week': {(df_c1['avg_last_week'] < 0).sum()}")
print(f"Negative values in 'avg_last_month': {(df_c1['avg_last_month'] < 0).sum()}")

CHECKOUT 1 SANITY CHECKS
Shape: (24, 6)

Null Values:
time                  0
today                 0
yesterday             0
same_day_last_week    0
avg_last_week         0
avg_last_month        0
dtype: int64

Data Types:
time                      str
today                   int64
yesterday               int64
same_day_last_week      int64
avg_last_week         float64
avg_last_month        float64
dtype: object

Expected 24 hourly rows: PASS
Duplicate time entries: 0
Negative values in 'today': 0
Negative values in 'yesterday': 0
Negative values in 'same_day_last_week': 0
Negative values in 'avg_last_week': 0
Negative values in 'avg_last_month': 0


### 3.2 Sanity Checks - Checkout 2

In [4]:
print("CHECKOUT 2 SANITY CHECKS")
print("=" * 40)
print(f"Shape: {df_c2.shape}")
print(f"\nNull Values:\n{df_c2.isnull().sum()}")
print(f"\nData Types:\n{df_c2.dtypes}")
print(f"\nExpected 24 hourly rows: {'PASS' if df_c2.shape[0] == 24 else 'FAIL'}")
print(f"Duplicate time entries: {df_c2['time'].duplicated().sum()}")
print(f"Negative values in 'today': {(df_c2['today'] < 0).sum()}")
print(f"Negative values in 'yesterday': {(df_c2['yesterday'] < 0).sum()}")
print(f"Negative values in 'same_day_last_week': {(df_c2['same_day_last_week'] < 0).sum()}")
print(f"Negative values in 'avg_last_week': {(df_c2['avg_last_week'] < 0).sum()}")
print(f"Negative values in 'avg_last_month': {(df_c2['avg_last_month'] < 0).sum()}")

CHECKOUT 2 SANITY CHECKS
Shape: (24, 6)

Null Values:
time                  0
today                 0
yesterday             0
same_day_last_week    0
avg_last_week         0
avg_last_month        0
dtype: int64

Data Types:
time                      str
today                   int64
yesterday               int64
same_day_last_week      int64
avg_last_week         float64
avg_last_month        float64
dtype: object

Expected 24 hourly rows: PASS
Duplicate time entries: 0
Negative values in 'today': 0
Negative values in 'yesterday': 0
Negative values in 'same_day_last_week': 0
Negative values in 'avg_last_week': 0
Negative values in 'avg_last_month': 0


### 3.3 Cross-Validation Between Snapshots

In [5]:
print("CROSS-VALIDATION")
print("=" * 40)
print(f"Same columns: {'PASS' if list(df_c1.columns) == list(df_c2.columns) else 'FAIL'}")
print(f"Same time entries: {'PASS' if list(df_c1['time']) == list(df_c2['time']) else 'FAIL'}")

print(f"\nCheckout 1 - Total Today: {df_c1['today'].sum()}")
print(f"Checkout 2 - Total Today: {df_c2['today'].sum()}")

print(f"\nCheckout 1 'yesterday' should match Checkout 2 'today' if consecutive days:")
match = (df_c1['yesterday'] == df_c2['today']).all()
print(f"  Result: {'MATCH' if match else 'NO MATCH'}")

print(f"\nCheckout 2 'yesterday' should match Checkout 1 'today' if consecutive days:")
match2 = (df_c2['yesterday'] == df_c1['today']).all()
print(f"  Result: {'MATCH' if match2 else 'NO MATCH'}")

print(f"\nCONCLUSION: Checkout 2 was captured the day AFTER Checkout 1.")
print(f"Checkout 2's 'yesterday' column matches Checkout 1's 'today' column perfectly.")

CROSS-VALIDATION
Same columns: PASS
Same time entries: PASS

Checkout 1 - Total Today: 526
Checkout 2 - Total Today: 427

Checkout 1 'yesterday' should match Checkout 2 'today' if consecutive days:
  Result: NO MATCH

Checkout 2 'yesterday' should match Checkout 1 'today' if consecutive days:
  Result: MATCH

CONCLUSION: Checkout 2 was captured the day AFTER Checkout 1.
Checkout 2's 'yesterday' column matches Checkout 1's 'today' column perfectly.


### 3.4 Consistency Check: Today vs Historical Baselines

In [ ]:
def compute_deviation(df, label):
    df = df.copy()
    df['hour'] = df['time'].str.replace('h', '').astype(int)
    df['dev_vs_yesterday'] = df['today'] - df['yesterday']
    df['dev_vs_last_week'] = df['today'] - df['same_day_last_week']
    df['dev_vs_avg_week'] = df['today'] - df['avg_last_week']
    df['dev_vs_avg_month'] = df['today'] - df['avg_last_month']

    df['pct_dev_avg_week'] = np.where(
        df['avg_last_week'] > 0,
        ((df['today'] - df['avg_last_week']) / df['avg_last_week']) * 100,
        0
    )

    print(f"\n{label} - Deviation Summary (Today vs Baselines)")
    print("=" * 60)

    business_hours = df[(df['hour'] >= 8) & (df['hour'] <= 22)]

    print(f"\nBusiness Hours (08h-22h):")
    print(f"  Total Today:           {business_hours['today'].sum()}")
    print(f"  Total Yesterday:       {business_hours['yesterday'].sum()}")
    print(f"  Total Same Day LW:     {business_hours['same_day_last_week'].sum()}")
    print(f"  Sum Avg Last Week:     {business_hours['avg_last_week'].sum():.1f}")
    print(f"  Sum Avg Last Month:    {business_hours['avg_last_month'].sum():.1f}")

    zero_hours_today = df[df['today'] == 0]
    if len(zero_hours_today) > 0:
        zero_during_business = zero_hours_today[(zero_hours_today['hour'] >= 8) & (zero_hours_today['hour'] <= 22)]
        if len(zero_during_business) > 0:
            print(f"\n  ALERT: Zero sales during business hours at: {list(zero_during_business['time'])}")
            for _, row in zero_during_business.iterrows():
                print(f"    {row['time']}: today=0, yesterday={row['yesterday']}, "
                      f"avg_week={row['avg_last_week']}, avg_month={row['avg_last_month']}")
        else:
            print(f"\n  Zero sales only during off-peak hours: {list(zero_hours_today['time'])}")

    large_drops = business_hours[business_hours['pct_dev_avg_week'] < -50]
    if len(large_drops) > 0:
        print(f"\n  ALERT: Hours with >50% drop vs weekly average:")
        for _, row in large_drops.iterrows():
            print(f"    {row['time']}: today={row['today']}, avg_week={row['avg_last_week']}, "
                  f"deviation={row['pct_dev_avg_week']:.1f}%")

    return df

df_c1_dev = compute_deviation(df_c1, "CHECKOUT 1")
df_c2_dev = compute_deviation(df_c2, "CHECKOUT 2")

## 4. SQL Query - Anomaly Detection

Using an in-memory SQLite database to simulate querying the checkout data for anomalies.
The query identifies hours where today's sales deviate significantly from the weekly average during business hours.

In [ ]:
conn = sqlite3.connect(':memory:')

df_c1_dev.to_sql('checkout_1', conn, index=False, if_exists='replace')
df_c2_dev.to_sql('checkout_2', conn, index=False, if_exists='replace')

query = """
SELECT
    'checkout_1' AS snapshot,
    time,
    hour,
    today,
    yesterday,
    same_day_last_week,
    avg_last_week,
    avg_last_month,
    ROUND(today - avg_last_week, 2) AS abs_deviation,
    CASE
        WHEN avg_last_week > 0 THEN ROUND(((today - avg_last_week) / avg_last_week) * 100, 1)
        ELSE 0
    END AS pct_deviation
FROM checkout_1
WHERE hour BETWEEN 8 AND 22
  AND (
      (avg_last_week > 0 AND ABS((today - avg_last_week) / avg_last_week) > 0.5)
      OR (today = 0 AND avg_last_week > 1)
  )

UNION ALL

SELECT
    'checkout_2' AS snapshot,
    time,
    hour,
    today,
    yesterday,
    same_day_last_week,
    avg_last_week,
    avg_last_month,
    ROUND(today - avg_last_week, 2) AS abs_deviation,
    CASE
        WHEN avg_last_week > 0 THEN ROUND(((today - avg_last_week) / avg_last_week) * 100, 1)
        ELSE 0
    END AS pct_deviation
FROM checkout_2
WHERE hour BETWEEN 8 AND 22
  AND (
      (avg_last_week > 0 AND ABS((today - avg_last_week) / avg_last_week) > 0.5)
      OR (today = 0 AND avg_last_week > 1)
  )
ORDER BY snapshot, hour
"""

df_anomalies = pd.read_sql_query(query, conn)
conn.close()

print("SQL QUERY RESULTS - Anomalous Hours (>50% deviation or zero sales during business hours)")
print("=" * 100)
print(df_anomalies.to_string(index=False))

## 5. Visualization

### 5.1 Checkout 1 - Normal Day

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

hours = df_c1_dev['hour']

axes[0].plot(hours, df_c1_dev['today'], 'o-', color='#2196F3', linewidth=2.5, markersize=6, label='Today')
axes[0].plot(hours, df_c1_dev['yesterday'], 's--', color='#FF9800', linewidth=1.5, markersize=4, label='Yesterday')
axes[0].plot(hours, df_c1_dev['same_day_last_week'], '^--', color='#4CAF50', linewidth=1.5, markersize=4, label='Same Day Last Week')
axes[0].fill_between(hours, df_c1_dev['avg_last_week'], alpha=0.15, color='#9C27B0', label='Avg Last Week')
axes[0].fill_between(hours, df_c1_dev['avg_last_month'], alpha=0.1, color='#607D8B', label='Avg Last Month')
axes[0].set_title('Checkout 1 (Normal Day) - Hourly Sales Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Checkouts')
axes[0].legend(loc='upper left', fontsize=9)
axes[0].grid(True, alpha=0.3)
axes[0].axvspan(8, 22, alpha=0.05, color='green', label='Business Hours')

hours2 = df_c2_dev['hour']

axes[1].plot(hours2, df_c2_dev['today'], 'o-', color='#F44336', linewidth=2.5, markersize=6, label='Today')
axes[1].plot(hours2, df_c2_dev['yesterday'], 's--', color='#FF9800', linewidth=1.5, markersize=4, label='Yesterday')
axes[1].plot(hours2, df_c2_dev['same_day_last_week'], '^--', color='#4CAF50', linewidth=1.5, markersize=4, label='Same Day Last Week')
axes[1].fill_between(hours2, df_c2_dev['avg_last_week'], alpha=0.15, color='#9C27B0', label='Avg Last Week')
axes[1].fill_between(hours2, df_c2_dev['avg_last_month'], alpha=0.1, color='#607D8B', label='Avg Last Month')

anomaly_mask = (df_c2_dev['today'] == 0) & (df_c2_dev['hour'] >= 8) & (df_c2_dev['hour'] <= 22)
if anomaly_mask.any():
    anomaly_hours = df_c2_dev[anomaly_mask]
    axes[1].scatter(anomaly_hours['hour'], anomaly_hours['today'], color='red', s=200, zorder=5,
                   marker='X', edgecolors='darkred', linewidths=2, label='ANOMALY (0 sales)')
    for _, row in anomaly_hours.iterrows():
        axes[1].annotate(f"OUTAGE\n{row['time']}",
                         xy=(row['hour'], 0),
                         xytext=(row['hour'], row['avg_last_week'] + 5),
                         fontsize=8, fontweight='bold', color='red', ha='center',
                         arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

axes[1].set_title('Checkout 2 (Anomalous Day) - Hourly Sales Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Number of Checkouts')
axes[1].set_xlabel('Hour of Day')
axes[1].legend(loc='upper left', fontsize=9)
axes[1].grid(True, alpha=0.3)
axes[1].axvspan(8, 22, alpha=0.05, color='green')
axes[1].set_xticks(range(24))
axes[1].set_xticklabels([f"{h:02d}h" for h in range(24)], rotation=45)

plt.tight_layout()
plt.savefig('checkout_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.2 Revenue Impact Estimation

In [ ]:
outage_hours = df_c2_dev[(df_c2_dev['today'] == 0) & (df_c2_dev['hour'] >= 8) & (df_c2_dev['hour'] <= 22)]

print("ESTIMATED IMPACT OF THE OUTAGE")
print("=" * 50)

expected_from_yesterday = outage_hours['yesterday'].sum()
expected_from_last_week = outage_hours['same_day_last_week'].sum()
expected_from_avg_week = outage_hours['avg_last_week'].sum()
expected_from_avg_month = outage_hours['avg_last_month'].sum()

print(f"\nOutage window: {list(outage_hours['time'].values)}")
print(f"Duration: {len(outage_hours)} hours")
print(f"\nExpected sales (based on different baselines):")
print(f"  Yesterday:          {expected_from_yesterday} checkouts")
print(f"  Same day last week: {expected_from_last_week} checkouts")
print(f"  Avg last week:      {expected_from_avg_week:.1f} checkouts")
print(f"  Avg last month:     {expected_from_avg_month:.2f} checkouts")

total_today_c2 = df_c2_dev['today'].sum()
total_without_outage = total_today_c2 + expected_from_avg_week

print(f"\nActual total sales (Checkout 2 day): {total_today_c2}")
print(f"Estimated total without outage:      {total_without_outage:.0f}")
print(f"Lost checkouts (est.):               {expected_from_avg_week:.0f}")
print(f"Revenue impact:                      {(expected_from_avg_week / total_without_outage * 100):.1f}% of daily sales")

### 5.3 Deviation Heatmap

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for idx, (df_dev, label, ax) in enumerate([
    (df_c1_dev, 'Checkout 1 (Normal)', axes[0]),
    (df_c2_dev, 'Checkout 2 (Anomalous)', axes[1])
]):
    deviations = df_dev[['dev_vs_yesterday', 'dev_vs_last_week', 'dev_vs_avg_week', 'dev_vs_avg_month']].T
    deviations.columns = df_dev['time']
    deviations.index = ['vs Yesterday', 'vs Last Week', 'vs Avg Week', 'vs Avg Month']

    vmax = max(abs(deviations.values.min()), abs(deviations.values.max()))
    im = ax.imshow(deviations.values, cmap='RdYlGn', aspect='auto', vmin=-vmax, vmax=vmax)

    ax.set_xticks(range(len(deviations.columns)))
    ax.set_xticklabels(deviations.columns, rotation=90, fontsize=8)
    ax.set_yticks(range(len(deviations.index)))
    ax.set_yticklabels(deviations.index, fontsize=9)
    ax.set_title(label, fontsize=12, fontweight='bold')

    plt.colorbar(im, ax=ax, shrink=0.8, label='Deviation (count)')

plt.tight_layout()
plt.savefig('checkout_deviation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Conclusions

### Data Consistency
- Both datasets have 24 rows (one per hour), no null values, no duplicate time entries, and no negative counts.
- The `yesterday` column in Checkout 2 matches the `today` column in Checkout 1, confirming they are consecutive days.

### Anomaly Detected in Checkout 2
- **Checkout 2 shows zero sales at 15h, 16h, and 17h** — three consecutive hours during peak business time.
- This is highly anomalous because:
  - Yesterday (Checkout 1) had 51, 41, and 45 sales at those same hours.
  - The weekly average for those hours is ~22, ~22, and ~18.
  - The monthly average for those hours is ~28, ~26, and ~23.
- **The pattern is consistent with a system outage** (e.g., payment gateway went down, checkout service crashed, or a bad deployment).

### Evidence Supporting an Outage
1. Sales **abruptly drop to exactly zero** — not a gradual decline, which rules out seasonal behavior.
2. The drop is **during the highest-traffic hours** of the day, where averages are at their peak.
3. Sales **resume at 18h** (13 checkouts), suggesting the system was restored, though at a reduced rate — possibly a recovery phase.
4. The **14h hour already shows a drop** (19 vs 32 yesterday, vs 35 same day LW) — the outage may have started during 14h.
5. Checkout 1 (the previous day) shows **no such anomaly**, confirming this is not a recurring pattern.

### Estimated Impact
- Approximately **62 lost checkouts** during the outage window (based on weekly average).
- This represents a significant percentage of the day's total sales.